# NNodely Documentation - Training

In [7]:
# uncomment the command below to install the nnodely package
#!pip install nnodely

from nnodely import *
import os

In [10]:
vehicle = nnodely(visualizer=TextVisualizer(),seed=2, workspace=os.path.join(os.getcwd(), 'results'))

# Dimensions of the layers
n  = 25
na = 21

#Create neural model inputs
velocity = Input('vel')
brake = Input('brk')
gear = Input('gear')
torque = Input('trq')
altitude = Input('alt',dimensions=na)
acc = Input('acc')

# Create neural network relations
air_drag_force = Linear(b=True)(velocity.last()**2)
breaking_force = -Relu(Fir(W_init = init_negexp, W_init_params={'size_index':0, 'first_value':0.002, 'lambda':3})(brake.sw(n)))
gravity_force = Linear(W_init=init_constant, W_init_params={'value':0}, dropout=0.1, W='gravity')(altitude.last())
fuzzi_gear = Fuzzify(6, range=[2,7], functions='Rectangular')(gear.last())
local_model = LocalModel(input_function=lambda: Fir(W_init = init_negexp, W_init_params={'size_index':0, 'first_value':0.002, 'lambda':3}))
engine_force = local_model(torque.sw(n), fuzzi_gear)

sum_rel = air_drag_force+breaking_force+gravity_force+engine_force

# Closing the loop
sum_rel.closedLoop(velocity)

# Create neural network output
out1 = Output('accelleration', sum_rel)
out2 = Output('velout', altitude.sw(2))

# Add the neural model to the nnodely structure and neuralization of the model
vehicle.addModel('acc',[out1,out2])
vehicle.addMinimize('acc_error', acc.last(), out1, loss_function='rmse')
vehicle.neuralizeModel(0.05)

data_struct = ['vel','trq','brk','gear','alt','acc']
data_folder = 'vehicle_data'
vehicle.loadData(name='dataset', source=data_folder, format=data_struct, skiplines=1)

[check_names] The name 'vel' is already in defined as NeuObj but it is overwritten.
[check_names] The name 'brk' is already in defined as NeuObj but it is overwritten.
[check_names] The name 'gear' is already in defined as NeuObj but it is overwritten.
[check_names] The name 'trq' is already in defined as NeuObj but it is overwritten.
[check_names] The name 'alt' is already in defined as NeuObj but it is overwritten.
[check_names] The name 'acc' is already in defined as NeuObj but it is overwritten.
[check_names] The name 'gravity' is already in defined as NeuObj but it is overwritten.
[check_names] The name 'accelleration' is already in defined as NeuObj but it is overwritten.
[check_names] The name 'velout' is already in defined as NeuObj but it is overwritten.
================================ nnodely Model =================================
{'Constants': {'Constant100': {'dim': 1, 'values': [2.0]}},
 'Functions': {'FFuzzify105': {'centers': [2.0, 3.0, 4.0, 5.0, 6.0, 7.0],
           

## Feed-forward Training

Use the 'trainModel' function to train a neuralized model. The function accept various arguments. you can decide which model to train (in case of a multi-model system) and what dataset to use.

The 'splits' argument will use the previously dataset loaded in the framework and divide it in [train, validation, test] according to the percentage

In [12]:
_ = vehicle.trainModel(models=['acc'], splits=[70,20,10])

======================== nnodely Model Train Parameters ========================
models:                       ['acc']
num of epochs:                100
update per epochs:            1
└>(n_samples-batch_size-prediction_samples+1)/(batch_size+step-1)+1
shuffle _data:                True
prediction samples:           0
step:                         0
closed loop:                  {}
connect:                      {}
train dataset:                None
	- num of samples:            53
	- batch size:                53
	- unused samples:            0
	  └>n_samples-prediction_samples-update_per_epochs*(batch_size+step-1)
val dataset:                  None
val {batch size, samples}:    {15, 15}
minimizers:                   {'acc_error': {'A': 'SamplePart191',
                                             'B': 'Add187',
                                             'loss': 'rmse'}}
optimizer:                    Adam
optimizer defaults:           {'lr': 0.001}
optimizer params:             [{'pa

you can specify various training parameters such as the shuffling, batch size, learning rate and number of epochs...

In [15]:
_ = vehicle.trainModel(models=['acc'], train_dataset='dataset', train_batch_size=8, num_of_epochs=10, lr=0.001, shuffle_data=True)

======================== nnodely Model Train Parameters ========================
models:                       ['acc']
num of epochs:                10
update per epochs:            9
└>(n_samples-batch_size-prediction_samples+1)/(batch_size+step-1)+1
shuffle _data:                True
prediction samples:           0
step:                         0
closed loop:                  {}
connect:                      {}
train dataset:                dataset
	- num of samples:            76
	- batch size:                8
	- unused samples:            4
	  └>n_samples-prediction_samples-update_per_epochs*(batch_size+step-1)
minimizers:                   {'acc_error': {'A': 'SamplePart191',
                                             'B': 'Add187',
                                             'loss': 'rmse'}}
optimizer:                    Adam
optimizer defaults:           {'lr': 0.001}
optimizer params:             [{'params': 'PFir101W'},
                               {'params': 'PFir107W'}

## Recurrent Training

The recurrent training can take place only when there are State variables, closed-loops or connections between inputs.

In case of a recurrent training, other parameters can be specified such as the number of prediction window. This is used to select for how many steps doing the recurrent loop

In [16]:
_ = vehicle.trainModel(models=['acc'], splits=[70,20,10], prediction_samples=10)

======================== nnodely Model Train Parameters ========================
models:                       ['acc']
num of epochs:                100
update per epochs:            1
└>(n_samples-batch_size-prediction_samples+1)/(batch_size+step-1)+1
shuffle _data:                True
prediction samples:           10
step:                         0
closed loop:                  {}
connect:                      {}
train dataset:                None
	- num of samples:            53
	- batch size:                43
	- unused samples:            0
	  └>n_samples-prediction_samples-update_per_epochs*(batch_size+step-1)
val dataset:                  None
val {batch size, samples}:    {5, 15}
minimizers:                   {'acc_error': {'A': 'SamplePart191',
                                             'B': 'Add187',
                                             'loss': 'rmse'}}
optimizer:                    Adam
optimizer defaults:           {'lr': 0.001}
optimizer params:             [{'pa

The 'step' size is used to decide whether to skip samples at each epoch. this will ensure a faster training time expecially when the prediction window is too high.

In [17]:
_ = vehicle.trainModel(models=['acc'], splits=[70,20,10], prediction_samples=10, step=5)

======================== nnodely Model Train Parameters ========================
models:                       ['acc']
num of epochs:                100
update per epochs:            0
└>(n_samples-batch_size-prediction_samples+1)/(batch_size+step-1)+1
shuffle _data:                True
prediction samples:           10
step:                         5
closed loop:                  {}
connect:                      {}
train dataset:                None
	- num of samples:            53
	- batch size:                43
	- unused samples:            43
	  └>n_samples-prediction_samples-update_per_epochs*(batch_size+step-1)
val dataset:                  None
val {batch size, samples}:    {5, 15}
minimizers:                   {'acc_error': {'A': 'SamplePart191',
                                             'B': 'Add187',
                                             'loss': 'rmse'}}
optimizer:                    Adam
optimizer defaults:           {'lr': 0.001}
optimizer params:             [{'p

The 'closed_loop' and 'connect' arguments are used if you want to do a recurrent loop only during training time without having the needs for your network to keep the state of the system inside a State variable

In [22]:
_ = vehicle.trainModel(models=['acc'], splits=[70,20,10], prediction_samples=10, step=5, connect={'vel': 'velout'})

======================== nnodely Model Train Parameters ========================
models:                       ['acc']
num of epochs:                100
update per epochs:            0
└>(n_samples-batch_size-prediction_samples+1)/(batch_size+step-1)+1
shuffle _data:                True
prediction samples:           10
step:                         5
closed loop:                  {}
connect:                      {'vel': 'velout'}
train dataset:                None
	- num of samples:            53
	- batch size:                43
	- unused samples:            43
	  └>n_samples-prediction_samples-update_per_epochs*(batch_size+step-1)
val dataset:                  None
val {batch size, samples}:    {5, 15}
minimizers:                   {'acc_error': {'A': 'SamplePart191',
                                             'B': 'Add187',
                                             'loss': 'rmse'}}
optimizer:                    Adam
optimizer defaults:           {'lr': 0.001}
optimizer params:  

RuntimeError: The expanded size of the tensor (1) must match the existing size (21) at non-singleton dimension 2.  Target sizes: [43, 1, 1].  Tensor sizes: [43, 1, 21]

In [25]:
_ = vehicle.trainModel(models=['acc'], splits=[70,20,10], prediction_samples=10, step=5, closed_loop={'acc':'accelleration'})

======================== nnodely Model Train Parameters ========================
models:                       ['acc']
num of epochs:                100
update per epochs:            0
└>(n_samples-batch_size-prediction_samples+1)/(batch_size+step-1)+1
shuffle _data:                True
prediction samples:           10
step:                         5
closed loop:                  {'acc': 'accelleration'}
connect:                      {}
train dataset:                None
	- num of samples:            53
	- batch size:                43
	- unused samples:            43
	  └>n_samples-prediction_samples-update_per_epochs*(batch_size+step-1)
val dataset:                  None
val {batch size, samples}:    {5, 15}
minimizers:                   {'acc_error': {'A': 'SamplePart191',
                                             'B': 'Add187',
                                             'loss': 'rmse'}}
optimizer:                    Adam
optimizer defaults:           {'lr': 0.001}
optimizer pa

## Set different weight for the minimization functions

use the 'minimize_gain' attribute to modify the importance of certain minimization functions by passing a dictionary with the gain factor for each minimizers.

In [27]:
_ = vehicle.trainModel(train_dataset='dataset', num_of_epochs=10, lr=0.001, shuffle_data=True, minimize_gain={'acc_error':2})

======================== nnodely Model Train Parameters ========================
models:                       ['acc']
num of epochs:                10
update per epochs:            1
└>(n_samples-batch_size-prediction_samples+1)/(batch_size+step-1)+1
shuffle _data:                True
prediction samples:           0
step:                         0
closed loop:                  {}
connect:                      {}
train dataset:                dataset
	- num of samples:            76
	- batch size:                76
	- unused samples:            0
	  └>n_samples-prediction_samples-update_per_epochs*(batch_size+step-1)
minimizers:                   {'acc_error': {'A': 'SamplePart191',
                                             'B': 'Add187',
                                             'gain': 2,
                                             'loss': 'rmse'}}
optimizer:                    Adam
optimizer defaults:           {'lr': 0.001}
optimizer params:             [{'params': 'PFir101W

## Early stopping and Best Model

Use one of the built-in early stopping and selection model functions or use a custom one. 

In [30]:
from nnodely.support.earlystopping import early_stop_patience, select_best_model
_ = vehicle.trainModel(train_dataset='dataset', num_of_epochs=10, lr=0.001, shuffle_data=True, early_stopping=early_stop_patience, select_model=select_best_model)

======================== nnodely Model Train Parameters ========================
models:                       ['acc']
num of epochs:                10
update per epochs:            1
└>(n_samples-batch_size-prediction_samples+1)/(batch_size+step-1)+1
shuffle _data:                True
early stopping:               early_stop_patience
early stopping params:        {}
prediction samples:           0
step:                         0
closed loop:                  {}
connect:                      {}
train dataset:                dataset
	- num of samples:            76
	- batch size:                76
	- unused samples:            0
	  └>n_samples-prediction_samples-update_per_epochs*(batch_size+step-1)
minimizers:                   {'acc_error': {'A': 'SamplePart191',
                                             'B': 'Add187',
                                             'loss': 'rmse'}}
optimizer:                    Adam
optimizer defaults:           {'lr': 0.001}
optimizer params:       